In [33]:
import pandas as pd
import math
import os.path
import time
import numpy as np
import talib
from talib import abstract
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
#backtest lib
from backtesting import Backtest
from backtesting.lib import SignalStrategy
from backtesting.lib import crossover

In [38]:
### API
binance_api_key = 'TESZdWxapwvZG90v1SJ3zMahcXySUkYv3bxRRKXKkfCMANtUS0Dw38iWALACQuHU'    #Enter your own API-key here
binance_api_secret = 'Uw1V3mbqoXDm9II7tC8lHRrYcGslHKhqo2bEXGelt08bnAn5lk4EbYfbKmHjWguY' #Enter your own API-secret here
### CONSTANTS ----add 15min
binsizes = {"1m": 1, "5m": 5,"15m":15, "1h": 60, "4h": 240, "1d": 1440}
batch_size = 750
# api instance
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

In [43]:
### get data funcs
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    # jsaw edit: not to get data from API if aiready exist data
    if os.path.isfile(filename): 
        print("read exist")
        data_df = pd.read_csv(filename)
        return data_df
    else: 
        data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): 
        print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: 
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: 
        data_df = data
    data_df.set_index('timestamp', inplace=True)
    data_df = data.astype(float)
    if save: 
        data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [45]:
#get data since 2017.7.1

df_4hr = get_all_binance('BTCUSDT', '4h',True)
df_1hr = get_all_binance('BTCUSDT', '1h',True)
df_15min = get_all_binance('BTCUSDT', '15m',True)
df_5min = get_all_binance('BTCUSDT', '5m', True)
df_1min = get_all_binance('BTCUSDT', '1m',True)

read exist
read exist
read exist
read exist
read exist


In [74]:
df = df_15min

In [36]:
class StrategyBollNew(SignalStrategy):
    
    def init(self):
        super().init()
        #rename to fit API
        df = self.getDf()
        
        strategyList = ['MACD','BBANDS']
        for critSymbol in strategyList:
            # calc criteria 
            output = eval('talib.abstract.'+ critSymbol +'(df)')
            # rename if  1-Dimention
            output.name = critSymbol.lower() if type(output) == pd.core.series.Series else None
            # merge result
            df = pd.concat([df,pd.DataFrame(output)],axis=1)
            #df = pd.merge(df, pd.DataFrame(output))
            
        # Precompute the two moving averages
        self.high = df['high']
        self.low = df['low']
        
        self.upper = df['upperband']
        self.lower = df['lowerband']
        
        signal_long = low < lower
        signal_short = high > upper
        
        signal = signal_long
        signal[signal_short] = -1
        self.set_signal(signal)
        
    def getDf(self):
        df = pd.DataFrame(columns=['open','high', 'low', 'close','volume'])
        df['open'] = pd.Series(self.data.Open)
        df['close'] = pd.Series(self.data.Close)
        df['high'] = pd.Series(self.data.High)
        df['low'] = pd.Series(self.data.Low)
        return df
        
    def next(self):
        if crossover(self.lower, self.low): #如果周線衝上月線，表示近期是上漲的，則買入
            self.buy()
        elif crossover(self.high, self.upper): #如果周線再與月線交叉，表示開始下跌了，則賣出
            self.sell()
        #super().next()

In [72]:
class StrategyMACD(SignalStrategy):
    
    def init(self):
        super().init()
        #get strategy data
        df = self.getDf()
        #EMA
        df['EMA5'] = talib.EMA(df['close'], timeperiod=5)
        df['EMA10'] = talib.EMA(df['close'], timeperiod=10)
        df['EMA20'] = talib.EMA(df['close'], timeperiod=20)
        df['EMA60'] = talib.EMA(df['close'], timeperiod=60)
        df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
        #MACD
        macdFast, macdSlow, histogram = talib.MACD(df['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
        df['MACD_DIF'] = macdFast
        df['MACD_DIM'] = macdSlow
        df['MACD_HIS'] = histogram
        #RSI
        df['RSI'] = talib.RSI(df['close'], timeperiod=30)
        print(df['RSI'])
        # cache to class for signal
        self.high = df['high']
        self.low = df['low']
        self.open = df['open']
        self.close = df['close']
        df['preClose'] = df['close'].shift(periods=1)

        signal_long = (self.close > df['EMA5']) & (df['EMA5'] > df['EMA10']) & (df['EMA10'] > df['EMA20']) & (df['EMA20'] > df['EMA60'])\
        & (df["MACD_DIF"] > df["MACD_DIM"]) & (df['preClose'] < df["EMA5"]) & (df['EMA60'] > df['EMA200'])
        #print(signal_long)
        
        signal_closePosition = (self.close < df['EMA5']) & (df['RSI'] < 30)
        #signal = signal_long
        #signal[signal_short] = -1
        self.set_signal(signal_long, signal_closePosition)
        
    def getDf(self):
        df = pd.DataFrame(columns=['open','high', 'low', 'close','volume'])
        df['open'] = pd.Series(self.data.Open)
        df['close'] = pd.Series(self.data.Close)
        df['high'] = pd.Series(self.data.High)
        df['low'] = pd.Series(self.data.Low)
        return df
        
    def next(self):
        super().next()

In [63]:
class StrategyBoll(SignalStrategy):
    
    def init(self):
        super().init()
        #rename to fit API
        df = self.getDf()
        df['EMA30'] = talib.EMA(df['close'], timeperiod=30)
        df['SMA30'] = talib.SMA(df['Close'], timeperiod=30)
        strategyList = ['MACD','BBANDS']
        for critSymbol in strategyList:
            # calc criteria 
            output = eval('talib.abstract.'+ critSymbol +'(df)')
            # rename if  1-Dimention
            output.name = critSymbol.lower() if type(output) == pd.core.series.Series else None
            # merge result
            df = pd.concat([df,pd.DataFrame(output)],axis=1)
            #df = pd.merge(df, pd.DataFrame(output))
            
        # Precompute the two moving averages
        self.high = df['high']
        self.low = df['low']
        
        self.upper = df['upperband']
        self.lower = df['lowerband']
        
        signal_long = self.low < self.lower
        signal_short = self.high > self.upper
        
        signal = signal_long
        signal[signal_short] = -1
        self.set_signal(self.low < self.lower, self.high > self.upper)
        
    def getDf(self):
        df = pd.DataFrame(columns=['open','high', 'low', 'close','volume'])
        df['open'] = pd.Series(self.data.Open)
        df['close'] = pd.Series(self.data.Close)
        df['high'] = pd.Series(self.data.High)
        df['low'] = pd.Series(self.data.Low)
        df
        return df
        
    def next(self):
        #if crossover(self.lower, self.low) and not self.position.is_long:
            #self.buy()
        #elif crossover(self.high, self.upper) and not self.position.is_short:
            #self.sell()
        super().next()

In [75]:
backtestData = df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close","volume":"Volume"})
backtestData = backtestData[["Open","High","Low","Close","Volume"]]
result1 = Backtest(backtestData, StrategyMACD, cash=100000)
print(result1.run())
#result1.plot()

<ipython-input-75-b4c1b15dc1dd>:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  result1 = Backtest(backtestData, StrategyMACD, cash=100000)


0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
113222    47.837563
113223    45.185911
113224    46.322097
113225    48.031121
113226    49.424229
Name: RSI, Length: 113227, dtype: float64
Start                                       0
End                                    113226
Duration                               113226
Exposure Time [%]                     68.0235
Equity Final [$]                       821623
Equity Peak [$]                        855330
Return [%]                            721.623
Buy & Hold Return [%]                 280.736
Max. Drawdown [%]                    -35.9833
Avg. Drawdown [%]                    -2.43503
Max. Drawdown Duration                  46776
Avg. Drawdown Duration                487.114
# Trades                                 2899
Win Rate [%]                          42.0835
Best Trade [%]                        139.009
Worst Trade [%]                       -27.784
Av